In [ ]:
#import torch
import cv2
import matplotlib.pyplot as plt
import os

# Define IoU threshold for matching predictions and ground truth labels
iou_threshold = 0.5

def load_ground_truth_labels(label_dir, frame_number):
    label_path = f'{label_dir}/gmbr_{frame_number}.txt'
    if not os.path.exists(label_path):
        return []  # Empty list if the label file doesn't exist

    with open(label_path, 'r') as f:
        lines = f.readlines()

    labels = []
    for line in lines:
        label = line.strip().split(' ')
        labels.append(label)

    return labels

def load_predicted_labels(predicted_label_dir, frame_number):
    label_path = f'{predicted_label_dir}/frame_{frame_number}.txt'
    if not os.path.exists(label_path):
        return []  # Empty list if the label file doesn't exist

    with open(label_path, 'r') as f:
        lines = f.readlines()

    labels = []
    for line in lines:
        label = line.strip().split(' ')
        labels.append(label)

    return labels

def calculate_iou(box1, box2):
    # Convert coordinates to float
    box1 = [float(x) for x in box1]
    box2 = [float(x) for x in box2]

    # Extract coordinates
    x1, y1, w1, h1 = box1
    x2, y2, w2, h2 = box2

    # Calculate intersection coordinates
    intersection_x1 = max(x1 - w1 / 2, x2 - w2 / 2)
    intersection_y1 = max(y1 - h1 / 2, y2 - h2 / 2)
    intersection_x2 = min(x1 + w1 / 2, x2 + w2 / 2)
    intersection_y2 = min(y1 + h1 / 2, y2 + h2 / 2)

    # Calculate intersection area
    intersection_area = max(0, intersection_x2 - intersection_x1) * max(0, intersection_y2 - intersection_y1)

    # Calculate union area
    box1_area = w1 * h1
    box2_area = w2 * h2
    union_area = box1_area + box2_area - intersection_area

    # Calculate IoU
    iou = intersection_area / union_area

    return iou

# Open the video file using OpenCV
def Evaluasi(ground_truth_label_dir,predicted_label_dir,video_number,model_number):
    path='validate0'+str(video_number)+'.mp4'
    video_path = path
    cap = cv2.VideoCapture(video_path)

    total_frames = int(cap.get(cv2.CAP_PROP_FRAME_COUNT))
    total_predictions = []
    ground_truth_counts = []
    frame_numbers = []

    true_positives = 0
    false_positives = 0
    false_negatives = 0
    TP1 = []  # List to store True Positives for each frame number
    FP1 = []  # List to store False Positives for each frame number
    FN1 = []  # List to store False Negatives for each frame number


    for frame_number in range(1,total_frames):
        ret, frame = cap.read()
        if not ret:
            break
        TP=0
        FP=0
        FN=0
        # Get image width and height
        ground_truth_labels = load_ground_truth_labels(ground_truth_label_dir, frame_number + 1)
        if 0<=frame_number<=total_frames and  ground_truth_labels:
            image_height, image_width = frame.shape[:2]
            predicted_labels = load_predicted_labels(predicted_label_dir, frame_number)

            total_predictions.append(len(predicted_labels))
            ground_truth_counts.append(len(ground_truth_labels))
            frame_numbers.append(frame_number)

        # Calculate the number of true positives, false positives, and false negatives
            predicted_boxes = [(float(label[1]), float(label[2]), float(label[3]), float(label[4])) for label in predicted_labels]
            ground_truth_boxes = [(float(label[1]), float(label[2]), float(label[3]), float(label[4])) for label in ground_truth_labels]

            predicted_matched = [False] * len(predicted_boxes)
            ground_truth_matched = [False] * len(ground_truth_boxes)

            for i, predicted_box in enumerate(predicted_boxes):
                for j, ground_truth_box in enumerate(ground_truth_boxes):
                    iou = calculate_iou(predicted_box, ground_truth_box)
                    if iou >= iou_threshold:
                        predicted_matched[i] = True
                        ground_truth_matched[j] = True
                        true_positives += 1
                        TP += 1

            false_positives += sum(not matched for matched in predicted_matched)
            false_negatives += sum(not matched for matched in ground_truth_matched)
            FP += sum(not matched for matched in predicted_matched)
            FN += sum(not matched for matched in ground_truth_matched)
            TP1.append(TP)
            FP1.append(FP)
            FN1.append(FN)
            
    cap.release()

    # Calculate precision, recall, and accuracy
    precision = true_positives / (true_positives + false_positives) if (true_positives + false_positives) != 0 else 0
    recall = true_positives / (true_positives + false_negatives) if (true_positives + false_negatives) != 0 else 0
    accuracy = true_positives / (true_positives + false_positives + false_negatives) if (true_positives + false_positives + false_negatives) != 0 else 0

    # Plotting the graph
    %matplotlib inline
    plt.plot(frame_numbers, total_predictions, label='Predicted')
    plt.plot(frame_numbers, TP1, label='True Positive')
    plt.plot(frame_numbers, FP1, label='False Positive')
    plt.plot(frame_numbers, FN1, label='False Negative')
    plt.plot(frame_numbers, ground_truth_counts, label='Ground Truth')
    plt.ylim(0, max(total_predictions)+1)
    plt.xlabel('Frame')
    plt.ylabel('Total Object Detections')
    plt.title('Hasil Video '+str(video_number)+' Model 4')
    plt.legend()    
    imagesave=predicted_label_dir+'.png'
    plt.savefig(imagesave)

    plt.show()

    # Print precision, recall, and accuracy
    print(f'validasi P-{model_number}-{video_number}')
    print(f'Precision: {precision:.2f}')
    print(f'Recall: {recall:.2f}')
    print(f'Accuracy: {accuracy:.2f}')
    print(frame_numbers)
    def calculate_total(arr):
        total = 0
        for num in arr:
            total += num
        return total

    # Example usage
    array_total = calculate_total(ground_truth_counts)
    array_total2 = calculate_total(total_predictions)
    print("The total is:", array_total2,"/",array_total)
for video_number in range(1,3):
    ground_truth_label_dir = 'validasi0'+str(video_number)
    for model_number in range(1,5):
        predicted_label_dir = 'P-'+str(model_number)+'-'+str(video_number)
        Evaluasi(ground_truth_label_dir,predicted_label_dir,video_number,model_number)


In [ ]:
#import torch
import cv2
import matplotlib.pyplot as plt
import os

# Define IoU threshold for matching predictions and ground truth labels
iou_threshold = 0.5

def load_ground_truth_labels(label_dir, frame_number):
    label_path = f'{label_dir}/frame_{frame_number:06d}.txt'
    if not os.path.exists(label_path):
        return []  # Empty list if the label file doesn't exist

    with open(label_path, 'r') as f:
        lines = f.readlines()

    labels = []
    for line in lines:
        label = line.strip().split(' ')
        labels.append(label)

    return labels

def load_predicted_labels(predicted_label_dir, frame_number):
    label_path = f'{predicted_label_dir}/frame_{frame_number}.txt'
    if not os.path.exists(label_path):
        return []  # Empty list if the label file doesn't exist

    with open(label_path, 'r') as f:
        lines = f.readlines()

    labels = []
    for line in lines:
        label = line.strip().split(' ')
        labels.append(label)

    return labels

def calculate_iou(box1, box2):
    # Convert coordinates to float
    box1 = [float(x) for x in box1]
    box2 = [float(x) for x in box2]

    # Extract coordinates
    x1, y1, w1, h1 = box1
    x2, y2, w2, h2 = box2

    # Calculate intersection coordinates
    intersection_x1 = max(x1 - w1 / 2, x2 - w2 / 2)
    intersection_y1 = max(y1 - h1 / 2, y2 - h2 / 2)
    intersection_x2 = min(x1 + w1 / 2, x2 + w2 / 2)
    intersection_y2 = min(y1 + h1 / 2, y2 + h2 / 2)

    # Calculate intersection area
    intersection_area = max(0, intersection_x2 - intersection_x1) * max(0, intersection_y2 - intersection_y1)

    # Calculate union area
    box1_area = w1 * h1
    box2_area = w2 * h2
    union_area = box1_area + box2_area - intersection_area

    # Calculate IoU
    iou = intersection_area / union_area

    return iou

# Open the video file using OpenCV
def Evaluasi(ground_truth_label_dir,predicted_label_dir,video_number,model_number):
    path='validate0'+str(video_number)+'.mp4'
    video_path = path
    cap = cv2.VideoCapture(video_path)

    total_frames = int(cap.get(cv2.CAP_PROP_FRAME_COUNT))
    total_predictions = []
    ground_truth_counts = []
    frame_numbers = []

    true_positives = 0
    false_positives = 0
    false_negatives = 0
    TP1 = []  # List to store True Positives for each frame number
    FP1 = []  # List to store False Positives for each frame number
    FN1 = []  # List to store False Negatives for each frame number


    for frame_number in range(1,total_frames):
        ret, frame = cap.read()
        if not ret:
            break
        TP=0
        FP=0
        FN=0
        # Get image width and height
        ground_truth_labels = load_ground_truth_labels(ground_truth_label_dir, frame_number + 1)
        if 0<=frame_number<=total_frames and  ground_truth_labels:
            image_height, image_width = frame.shape[:2]
            predicted_labels = load_predicted_labels(predicted_label_dir, frame_number)

            total_predictions.append(len(predicted_labels))
            ground_truth_counts.append(len(ground_truth_labels))
            frame_numbers.append(frame_number)

        # Calculate the number of true positives, false positives, and false negatives
            predicted_boxes = [(float(label[1]), float(label[2]), float(label[3]), float(label[4])) for label in predicted_labels]
            ground_truth_boxes = [(float(label[1]), float(label[2]), float(label[3]), float(label[4])) for label in ground_truth_labels]

            predicted_matched = [False] * len(predicted_boxes)
            ground_truth_matched = [False] * len(ground_truth_boxes)

            for i, predicted_box in enumerate(predicted_boxes):
                for j, ground_truth_box in enumerate(ground_truth_boxes):
                    iou = calculate_iou(predicted_box, ground_truth_box)
                    if iou >= iou_threshold:
                        predicted_matched[i] = True
                        ground_truth_matched[j] = True
                        true_positives += 1
                        TP += 1

            false_positives += sum(not matched for matched in predicted_matched)
            false_negatives += sum(not matched for matched in ground_truth_matched)
            FP += sum(not matched for matched in predicted_matched)
            FN += sum(not matched for matched in ground_truth_matched)
            TP1.append(TP)
            FP1.append(FP)
            FN1.append(FN)
            
    cap.release()

    # Calculate precision, recall, and accuracy
    precision = true_positives / (true_positives + false_positives) if (true_positives + false_positives) != 0 else 0
    recall = true_positives / (true_positives + false_negatives) if (true_positives + false_negatives) != 0 else 0
    accuracy = true_positives / (true_positives + false_positives + false_negatives) if (true_positives + false_positives + false_negatives) != 0 else 0

    # Plotting the graph
    %matplotlib inline
    plt.plot(frame_numbers, total_predictions, label='Predicted')
    plt.plot(frame_numbers, TP1, label='True Positive')
    plt.plot(frame_numbers, FP1, label='False Positive')
    plt.plot(frame_numbers, FN1, label='False Negative')
    plt.plot(frame_numbers, ground_truth_counts, label='Ground Truth')
    plt.ylim(0, max(total_predictions)+1)
    plt.xlabel('Frame')
    plt.ylabel('Total Object Detections')
    plt.title('Hasil Video '+str(video_number)+' Model 4')
    plt.legend()    
    imagesave=predicted_label_dir+'.png'
    plt.savefig(imagesave)

    plt.show()

    # Print precision, recall, and accuracy
    print(f'validasi P-{model_number}-{video_number}')
    print(f'Precision: {precision:.2f}')
    print(f'Recall: {recall:.2f}')
    print(f'Accuracy: {accuracy:.2f}')
    print(frame_numbers)
    def calculate_total(arr):
        total = 0
        for num in arr:
            total += num
        return total

    # Example usage
    array_total = calculate_total(ground_truth_counts)
    array_total2 = calculate_total(total_predictions)
    print("The total is:", array_total2,"/",array_total)
for video_number in range(3,5):
    ground_truth_label_dir = 'validasi0'+str(video_number)
    for model_number in range(1,5):
        predicted_label_dir = 'P-'+str(model_number)+'-'+str(video_number)
        Evaluasi(ground_truth_label_dir,predicted_label_dir,video_number,model_number)
